In [ ]:
!pip install contextily
!pip install pyrsgis

In [ ]:
import contextily as cx
from ipywidgets import interact
from math import floor
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
from pyrsgis import raster
from pyrsgis.ml import imageChipsFromArray
import random
import seaborn as sns
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from statistics import mean
import tensorflow as tf
import rasterio
import torch
from torchvision import datasets, transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# file names
feature_file = '/content/drive/MyDrive/Nadi_data_new/2013_original.tif'

In [ ]:
# read the image
dsFeatures, arrFeatures = raster.read(feature_file, bands='all')
min_x = dsFeatures.bbox[0][0]
max_x = dsFeatures.bbox[1][0]
min_y = dsFeatures.bbox[0][1]
max_y = dsFeatures.bbox[1][1]

extent = [min_x, max_x, min_y, max_y]

In [ ]:
extent

In [ ]:
arrFeatures.shape

In [ ]:
def plot_array_stretched(array, label, n):
    fig, ax = plt.subplots(figsize=(10, 10))
    v_mean = np.array(np.nanmean(array))
    v_std = np.array(np.nanstd(array))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.f'))
    plt.yticks(rotation=90, va='center')
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.f'))
    cb = ax.imshow(array, cmap='Greys_r', extent=extent, vmin=v_mean-(n*v_std), vmax=v_mean+(n*v_std))
    divider = make_axes_locatable(ax)
    cax = divider.new_vertical(size='5%', pad=0.5, pack_start=True)
    fig.add_axes(cax)
    plt.colorbar(cb, orientation='horizontal', label=label, cax=cax)
    plt.grid(False)
    plt.show()

def plot_array(array, label):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.f'))
    plt.yticks(rotation=90, va='center')
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.f'))
    cb = ax.imshow(array, cmap='Greys_r', extent=extent)
    divider = make_axes_locatable(ax)
    cax = divider.new_vertical(size='5%', pad=0.5, pack_start=True)
    fig.add_axes(cax)
    plt.colorbar(cb, orientation='horizontal', label=label, cax=cax)
    plt.grid(False)
    plt.show()

@interact(band=range(1, arrFeatures.shape[0]+1))
def show_dist(band):
    plot_array_stretched(arrFeatures[band-1, :, :], f'Band {band}', 3)

In [ ]:
np.count_nonzero(np.isnan(arrFeatures))

**Chip**

In [ ]:
import copy
from copy import deepcopy
from sklearn.feature_extraction import image

In [ ]:
def imageChipsFromSingleBandArray(data_arr, y_size=5, x_size=5):
    image_chips = deepcopy(data_arr)
    image_chips = np.pad(image_chips, (int(y_size/2),int(x_size/2)), 'reflect')
    image_chips = image.extract_patches_2d(image_chips, (y_size, x_size))

    return(image_chips)
def imageChipsFromArray_update(data_array, x_size=5, y_size=5):

    # if array is a single band image
    if len(data_array.shape) == 2:
        return(imageChipsFromSingleBandArray(data_array, x_size=x_size, y_size=y_size))

    # if array is a multi band image
    elif len(data_array.shape) > 2:
        data_array = copy.copy(data_array)
        data_array = np.rollaxis(data_array, 0, 3)

        for band in range(data_array.shape[2]):
            temp_array = imageChipsFromSingleBandArray(data_array[:, :, band], x_size=x_size, y_size=y_size)

            if band == 0:
                out_array = np.expand_dims(temp_array, axis=3)
            else:
                out_array = np.concatenate((out_array, np.expand_dims(temp_array, axis=3)), axis=3)

        return(out_array)

    # if shape of the image is less than two dimensions, raise error
    else:
        raise Exception("Sorry, only two or three dimensional arrays allowed.")

In [ ]:
# normalize the image and generate chips
arrFeatures_fuzzy = np.zeros(arrFeatures.shape)

for i in range(arrFeatures.shape[0]):
    bandMin = arrFeatures[i, :, :].min()
    bandMax = arrFeatures[i, :, :].max()
    bandRange = bandMax-bandMin
    for j in range(arrFeatures.shape[1]):
        for k in range(arrFeatures.shape[2]):
            arrFeatures_fuzzy[i, j, k] = (arrFeatures[i, j, k]-bandMin)/bandRange
#create chips using pyrsgis
#features_chips = imageChipsFromArray(arrFeatures_fuzzy, x_size=7, y_size=7)
features_chips = imageChipsFromArray_update(arrFeatures_fuzzy, x_size=9, y_size=9)

In [ ]:
780*818

In [ ]:
features_chips.shape

**Loading the training dataset (manual)**

In [ ]:
positiveLabel_file = '/content/drive/MyDrive/Nadi_data_new/Label_2013_Nadi_1750_samples.tif'

In [ ]:
_dsPositiveLabels, arrPositiveLabels = raster.read(positiveLabel_file,bands='all')


In [ ]:
arrPositiveLabels.shape

In [ ]:
np.unique(arrPositiveLabels)

In [ ]:
n_class_pos = len(np.unique(arrPositiveLabels))

In [ ]:
n_class_pos

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.yticks(rotation=90, va='center')
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
cmap = mpl.colors.ListedColormap(["white",'red','lightgreen','darkgreen','yellow','blue','lightyellow','pink'])
image = plt.imshow(arrPositiveLabels, cmap=cmap, extent=extent, interpolation='nearest')
divider = make_axes_locatable(ax)
cax = divider.new_vertical(size='5%', pad=0.5, pack_start=True)
fig.add_axes(cax)
cbar = plt.colorbar(image, orientation='horizontal', label='Alteration Type', cax=cax)
tick_locs = (np.arange(n_class_pos)-0.5)*(n_class_pos-1)/n_class_pos
cbar.set_ticks(tick_locs)
# Urban Areas, Grass/Agricultural Land, Forest, Bare Soil, Water Bodies, Coastal Areas
cbar.set_ticklabels(["Background",'Urban Areas','Grass/Agricultural Land','Forest','Bare Soil','Water Bodies','Coastal Areas','Alluvium'])
plt.grid(False)

**Sampling**

In [ ]:
arrPositiveLabels=arrPositiveLabels+1
np.unique(arrPositiveLabels)

In [ ]:
# generate random samples
nonZero_count = np.count_nonzero(arrPositiveLabels)
index = np.transpose(np.where(arrPositiveLabels==0))
index = index.astype(int)
randomIndex = random.sample(range(index.shape[0]), int(nonZero_count/(n_class_pos-1)))
arrNegativeLabels = np.zeros(arrPositiveLabels.shape)

for i in range(len(randomIndex)):
    arrNegativeLabels[index[randomIndex[i], 0], index[randomIndex[i], 1]] = n_class_pos # max value of labels plus one which equals n_class

arrPositiveLabels_flat = arrPositiveLabels.flatten()
arrNegativeLabels_flat = arrNegativeLabels.flatten()
# separate and balance the classes
positiveFeatures = features_chips[arrPositiveLabels_flat!=0]
positiveLabels = arrPositiveLabels_flat[arrPositiveLabels_flat!=0]


#negativeFeatures = features_chips[arrNegativeLabels_flat==n_class_pos]
#negativeLabels = arrNegativeLabels_flat[arrNegativeLabels_flat==n_class_pos]
# combine the balanced features
# features = np.concatenate((positiveFeatures, negativeFeatures), axis=0)
# labels = np.concatenate((positiveLabels, negativeLabels), axis=0)
# labels[labels==n_class_pos] = 0

#Exclude the background
features=positiveFeatures
labels=positiveLabels

In [ ]:
len(features_chips[arrPositiveLabels_flat==6])

In [ ]:
temp = 250-91

In [ ]:
from sklearn.utils import resample
coastal_upsample = resample(features_chips[arrPositiveLabels_flat==6],
             replace=True,
             n_samples=temp,
             random_state=42)

In [ ]:
coastal_upsample.shape

In [ ]:
arr = np.concatenate((features, coastal_upsample), axis=0)

In [ ]:
features=arr

In [ ]:
features.shape

In [ ]:
coastal_label=[6]*temp

In [ ]:
type(coastal_label)

In [ ]:
new_label=np.concatenate((labels,coastal_label),axis=0)

In [ ]:
labels=new_label

In [ ]:
np.unique(labels)

In [ ]:
labels=labels-1

In [ ]:
np.unique(labels)

In [ ]:
n_class_pos = len(np.unique(labels))
print(n_class_pos)

In [ ]:
labels_df=pd.DataFrame()
labels_df['labels']=labels
labels_df['labels'].value_counts()

SPLIT

*****

In [ ]:
# # define a function to split features and labels
def train_test_split(features, labels, n_class_pos=n_class_pos, trainProp=0.7):
    labels_df=pd.DataFrame()
    labels_df['labels']=labels
    labels_df.reset_index(inplace=True)
    labels_df['labels']=labels_df['labels'].astype(int)

    train_list=[]
    test_list=[]
    for class_index in range(n_class_pos):
      sub_labels_df=labels_df[labels_df['labels'] == class_index ]
      dataSize=len(sub_labels_df['index'])
      sliceIndex = int(dataSize*trainProp)
      randIndex = list(sub_labels_df['index'])
      #random.seed(45)
      random.shuffle(randIndex)

      sub_train_list=randIndex[:sliceIndex]
      train_list = train_list + sub_train_list

      sub_test_list=randIndex[sliceIndex:]
      test_list = test_list + sub_test_list

    train_x = features[[train_list], :, :, :][0]
    test_x = features[[test_list], :, :, :][0]
    train_y = labels[train_list]
    test_y = labels[test_list]
    return(train_x, train_y, test_x, test_y)

# call the function to split the dataset
train_x, train_y, test_x, test_y = train_test_split(features, labels)

In [ ]:
features.shape

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
# Label_aside file use random.seed(35)
# 1106 Label file use random.seed(45)
train_y_df=pd.DataFrame()
train_y_df['labels']=train_y
train_y_df['labels'].value_counts()

In [ ]:
test_y_df=pd.DataFrame()
test_y_df['labels']=test_y
test_y_df['labels'].value_counts()

**MODEL**

In [ ]:
def roc_auc(model, test_x, test_y, n_classes):
    roc_auc = []
    test_z = model.predict(test_x)
    test_y_dummies = pd.get_dummies(test_y).values

    for i in range(n_classes):
        roc_auc.append(roc_auc_score(test_y_dummies[:, i], test_z[:, i]))

    return roc_auc

# plot any graph relating to any model
def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics)+1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and Testing '+metric.capitalize())
    plt.xlabel('Epochs')
    plt.ylabel(metric.capitalize())
    plt.legend(['Training '+metric.capitalize(), 'Testing '+metric.capitalize()])
    plt.show()

def roc_plot(test_y, test_z, n_classes, labels_name, average='macro'):
    fpr = {}
    tpr = {}
    roc_auc = {}

    test_y_dummies = pd.get_dummies(test_y).values

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(test_y_dummies[:, i], test_z[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots()
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver Operating Characteristic')

    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='{}, AUC = {}'.format(labels_name[i], '{0:.4f}'.format(roc_auc[i])))

    ax.legend(loc='best')
    ax.grid(alpha=0.5)
    sns.despine()
    plt.show()
    print('ROC AUC score:', roc_auc_score(test_y_dummies, test_z, average=average))

best_model = tf.keras.models.Sequential()
best_accuracy = 0
best_history = None
###*********************************#######################
experimental_runs = 10
cv = 10
#'Urban','Crop/Grass','Trees','Bare','Water','Coastal']
metrics_columns = ['Accuracy', 'Precision', 'Recall', 'F1 Score','AUC_Urban Areas','AUC_Grass/Agricultural Land','AUC_Forest','AUC_Bare Soil','AUC_Water Bodies','AUC_Coastal Areas','AUC_Alluvium','Roc_auc_mean']
temp_metrics_df = pd.DataFrame(np.zeros((cv, len(metrics_columns))), columns=metrics_columns)

for i in range(experimental_runs):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=1, padding='valid', activation='relu', input_shape=(train_x.shape[1], train_x.shape[2], train_x.shape[3])))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Conv2D(48, kernel_size=1, padding='valid', activation='relu'))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Conv2D(64, kernel_size=1, padding='valid', activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(80, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(7, activation='softmax')) #Change the number of last layer based on the output types =====> the categories is 5

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    ###*********************************#######################
    history = model.fit(train_x, train_y, epochs=100, validation_data=(test_x, test_y))

    for j in range(cv):
        train_x, train_y, test_x, test_y = train_test_split(features, labels)
        # predict for the test dataset
        test_z = model.predict(test_x)
        # calculate and display error metrics
        test_z_class = test_z.argmax(axis=1)

        aScore = accuracy_score(test_y, test_z_class)
        pScore = precision_score(test_y, test_z_class, average='macro')
        rScore = recall_score(test_y, test_z_class, average='macro')
        fScore = f1_score(test_y, test_z_class, average='macro')
        cMatrix = confusion_matrix(test_y, test_z_class)
        metrics_array = [aScore, pScore, rScore, fScore]
        roc_auc_class = roc_auc(model, test_x, test_y, n_class_pos)

        for k in range(n_class_pos):
            metrics_array.append(roc_auc_class[k])

        roc_auc_mean = mean(roc_auc_class)
        metrics_array.append(roc_auc_mean)
        temp_metrics_df.loc[j] = metrics_array

        if j==cv-1:
            # accuracy can be replaced with other scores
            aScore_avg = temp_metrics_df['Accuracy'].mean()

            if best_accuracy < aScore_avg:
                best_model = model
                best_accuracy_avg = aScore_avg
                best_history = history

print(f'\nBest Average Accuracy:\n{best_accuracy_avg}')

test_z = best_model.predict(test_x)
test_z_class = test_z.argmax(axis=1)
aScore = accuracy_score(test_y, test_z_class)
pScore = precision_score(test_y, test_z_class, average='macro')
rScore = recall_score(test_y, test_z_class, average='macro')
fScore = f1_score(test_y, test_z_class, average='macro')
cMatrix = confusion_matrix(test_y, test_z_class)
metrics_array = [aScore, pScore, rScore, fScore]
roc_auc_class = roc_auc(best_model, test_x, test_y, n_class_pos)
metrics_array.extend(roc_auc_class)
roc_auc_mean = mean(roc_auc_class)
metrics_array.append(roc_auc_mean)

print(f'\nConfusion Matrix:\n{cMatrix}')
print(f'\nAccuracy: {aScore}')
print(f'\nPrecision: {pScore}')
print(f'\nRecall: {rScore}')
print(f'\nF1 Score: {fScore}')

plot_metric(history, 'loss')
print('')
plot_metric(history, 'accuracy')
labels_name = ['Urban Areas','Grass/Agricultural Land','Forest','Bare Soil','Water Bodies','Coastal Areas','Alluvium'] #Labels name should be the same with the class
roc_plot(test_y, test_z, n_class_pos, labels_name)


In [ ]:
features_chips.shape

In [ ]:
features_chips_prediction_Nadi_2013=best_model.predict(features_chips)
features_chips_prediction_Nadi_2013.shape

In [ ]:
best_model.save('/content/drive/MyDrive/Nadi_data_new2/Nadi_2013_model_700_samples.keras')

In [ ]:
features_chips_class_Nadi_2013 = features_chips_prediction_Nadi_2013.argmax(axis=1)
features_chips_class_Nadi_2013.shape

In [ ]:
features_chips_prediction_arr_Nadi_2013=features_chips_class_Nadi_2013.reshape((780, 818))
features_chips_prediction_arr_Nadi_2013.shape

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.f'))
plt.yticks(rotation=90, va='center')
ax.xaxis.set_major_formatter(FormatStrFormatter('%.f'))
cmap = mpl.colors.ListedColormap(['red','lightgreen','darkgreen','yellow','blue','lightyellow','gray'])
image = plt.imshow(features_chips_prediction_arr_Nadi_2013, cmap=cmap, extent=extent, interpolation='nearest')
divider = make_axes_locatable(ax)
cax = divider.new_vertical(size='5%', pad=0.5, pack_start=True)
fig.add_axes(cax)
cbar = plt.colorbar(image, orientation='horizontal', label='Alteration Type', cax=cax)
tick_locs = (np.arange(n_class_pos) +0.5)*(n_class_pos-1)/n_class_pos
cbar.set_ticks(tick_locs)
cbar.set_ticklabels(['Urban','Crop/Grass','Trees','Bare','Water','Coastal','Alluvium'])
plt.grid(False)


##########################################
# Algorthm: CNN
# Location: Nadi
# Time: 2013
##########################################

This is 2013 image

*****

In [ ]:
# 2013 Array
features_chips_prediction_arr_Nadi_2013

In [ ]:
features_chips_prediction_arr_Nadi_2013.shape

In [ ]:

Urban_Nadi_2013 = np.where(Mapped_Nadi_2013_arr==0,1,0)

print(f"Image size: {Urban_Nadi_2013.shape}")
print(f"np.count_nonzero(Urban_Nadi_2013): {np.count_nonzero(Urban_Nadi_2013)}")

In [ ]:
24946/(795*835)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(Urban_Nadi_2013)